# **Pretrained Model for Feature Exraction**

In [1]:
import zipfile

with zipfile.ZipFile('/content/dataset.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [2]:
import os
from PIL import Image
import numpy as np
import keras
import cv2
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
import random

In [3]:
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications.xception import Xception
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.inception_v3 import InceptionV3
from keras.applications.densenet import DenseNet121

vggModel = VGG16(weights='imagenet', include_top=False)
resModel = ResNet50(weights='imagenet',include_top=False)
xceptionModel = Xception(weights='imagenet',include_top=False)
mobilenetModel = MobileNetV2(weights='imagenet',include_top=False)
inceptionModel = InceptionV3(weights='imagenet',include_top=False)
densenetModel = DenseNet121(weights='imagenet',include_top=False)


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


/tmp/ipython-input-3-2349665724.py:11: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  mobilenetModel = MobileNetV2(weights='imagenet',include_top=False)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
model = resModel
X = []
Y = []
classCount = 0

basepath='./dataset'
for folder in os.scandir(basepath):
    if folder.is_dir():
        for imfile in os.scandir(folder.path):
            image = Image.open(imfile.path)
            image = np.array(image.resize((224,224)))
            image = image.reshape([-1,224,224,3])
            feature=(model.predict([image])).flatten()
            X.append(feature)
            Y.append(folder.name)
        classCount+=1
X = np.array(X)

/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor_19
Received: inputs=('Tensor(shape=(1, 224, 224, 3))',)
  warnings.warn(msg)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 322ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step


In [5]:
print(np.shape(X))
print(np.shape(Y))
print(Y)

(22, 100352)
(22,)
['melon', 'melon', 'melon', 'melon', 'melon', 'melon', 'melon', 'melon', 'melon', 'melon', 'lemon', 'lemon', 'lemon', 'lemon', 'lemon', 'lemon', 'lemon', 'lemon', 'lemon', 'lemon', 'lemon', 'lemon']


In [6]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(X, Y)
model = SelectFromModel(clf, prefit=True)
X = model.transform(X)

print(np.shape(X))
print(np.shape(Y))

(22, 151)
(22,)


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

encoder = LabelEncoder()
y = tf.keras.utils.to_categorical(encoder.fit_transform(Y))

X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(y), test_size=0.3, random_state=42)

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
dnnModel=models.Sequential()
dnnModel.add(layers.Dense(10000,activation="relu",input_shape=(np.shape(X)[1],)))
dnnModel.add(layers.Dense(5000,activation="relu"))
dnnModel.add(layers.Dense(2000,activation="relu"))
dnnModel.add(layers.Dense(500,activation="relu"))
dnnModel.add(layers.Dense(300,activation="relu"))
dnnModel.add(layers.Dense(200,activation="relu"))
dnnModel.add(layers.Dense(120,activation="relu"))
dnnModel.add(layers.Dense(30,activation="relu"))
dnnModel.add(layers.Dense(30,activation="relu"))
dnnModel.add(layers.Dense(classCount,activation="softmax"))
dnnModel.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_30 (Dense)                │ (None, 10000)          │     1,560,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 5000)           │    50,005,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ (None, 2000)           │    10,002,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 500)            │     1,000,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 300)            │       150,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 200)            │        60,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 120)            │        24,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 30)             │         3,630 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (None, 30)             │           930 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ (None, 2)              │            62 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 62,806,742 (239.59 MB)

 Trainable params: 62,806,742 (239.59 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
dnnModel.compile(optimizer="Adam",loss="categorical_crossentropy",metrics=["accuracy"])

dnnModel.fit(X_train,y_train,epochs=25,batch_size=8)

testloss, testAccuracy=dnnModel.evaluate(X_test,y_test)

print(testAccuracy)

NameError: name 'dnnModel' is not defined

# **Fine Tuning Pretrained Model**

In [ ]:
X_images = []
Y_images = []

basepath='./dataset'
for folder in os.scandir(basepath):
    if folder.is_dir():
        for imfile in os.scandir(folder.path):
            image = Image.open(imfile.path).convert('RGB') # Ensure image is RGB
            image = image.resize((224,224))
            image_array = np.array(image)
            X_images.append(image_array)
            Y_images.append(folder.name)

X_images = np.array(X_images)
Y_images = np.array(Y_images)

# Normalize image data (optional but recommended)
X_images = X_images / 255.0

In [ ]:
# Encode labels
encoder = LabelEncoder()
y_encoded = tf.keras.utils.to_categorical(encoder.fit_transform(Y_images))
print(y_encoded)
# Split data
X_train_ft, X_test_ft, y_train_ft, y_test_ft = train_test_split(X_images, y_encoded, test_size=0.3, random_state=42)

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [ ]:
# Assuming you want to fine-tune the ResNet50 model
# Load the ResNet50 model without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Create a new model on top of the base model
fine_tune_model = models.Sequential()
fine_tune_model.add(base_model)
fine_tune_model.add(layers.GlobalAveragePooling2D()) # Add a pooling layer
fine_tune_model.add(layers.Dense(256, activation='relu')) # Add dense layers
fine_tune_model.add(layers.Dropout(0.5)) # Add dropout for regularization
fine_tune_model.add(layers.Dense(classCount, activation='softmax')) # Output layer with the number of classes

fine_tune_model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_42 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,846,530 (56.64 MB)

 Trainable params: 131,842 (515.01 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [ ]:
# Compile the fine-tune model
# Use a smaller learning rate for fine-tuning
fine_tune_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                         loss='categorical_crossentropy',
                         metrics=['accuracy'])


# Train the fine-tune model
fine_tune_model.fit(X_train_ft, y_train_ft, epochs=25, batch_size=32, validation_data=(X_test_ft, y_test_ft))

# Evaluate the fine-tune model
test_loss_ft, test_accuracy_ft = fine_tune_model.evaluate(X_test_ft, y_test_ft)

print(f"Fine-tuned model test accuracy: {test_accuracy_ft}")

Epoch 1/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step - accuracy: 0.2667 - loss: 0.8029 - val_accuracy: 0.4286 - val_loss: 0.7158
Epoch 2/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 0.4667 - loss: 0.7711 - val_accuracy: 0.5714 - val_loss: 0.6672
Epoch 3/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 483ms/step - accuracy: 0.6667 - loss: 0.6080 - val_accuracy: 0.5714 - val_loss: 0.6279
Epoch 4/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step - accuracy: 0.8000 - loss: 0.3924 - val_accuracy: 0.5714 - val_loss: 0.5650
Epoch 5/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step - accuracy: 0.6667 - loss: 0.5048 - val_accuracy: 0.7143 - val_loss: 0.5080
Epoch 6/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step - accuracy: 0.7333 - loss: 0.4540 - val_accuracy: 0.8571 - val_loss: 0.4664
Epoch 7/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step - accuracy: 0.8000 - loss: 0.3628 - val_accuracy: 0.8571 - val_loss: 0.4300
Epoch 8/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step - accuracy: 0.9333 - loss: 0.2536 - val_accuracy: 1.0000 - val_loss: 0.3